In [1]:
import json 
import time
import aiohttp
import asyncio
import requests 
import pandas as pd 
import altair as alt 

In [2]:
def compose_url(base_url: str, parameters: dict, page: int = 1) -> str:
    """Helper method to compose the API url with parameters"""
    url = base_url + "?" if not base_url.endswith("?") else base_url
    params = "&".join([f"{p}={v}" for p,v in parameters.items()]) + f"&page={page}"
    return url + params


In [3]:
async def fetch_data(session: aiohttp.ClientSession, url: str):
    """Uses ClientSession to create the async call to the API"""
    async with session.get(url) as response:
        return await response.json()

async def attempt_delay(session: aiohttp.ClientSession, url: str):
    """The request may fail. This method adds a delay to the failing requests"""
    try:
        return await fetch_data(session, url)
    except Exception as e:
        await asyncio.sleep(0.2)
        return await attempt_delay(session, url)

In [4]:
async def get(base_url: str, parameters: dict) -> list:
    st = time.time()
    result = []
    tasks = []
    async with aiohttp.ClientSession() as session:
        url = compose_url(base_url, parameters)
        data = await attempt_delay(session, url)
        total_pages = data["pagination"]["total_pages"]
        result.extend(data["items"])

        for page in range(1, total_pages + 1):
            url = compose_url(base_url, parameters, page)
            tasks.append(attempt_delay(session, url))

        responses = await asyncio.gather(*tasks)
        for resp in responses:
            result.extend(resp["items"])
            
    et = time.time()
    print(f"Took {et-st:.6f} seconds")
    return result

### Carregando previsões: 

In [5]:

def get_predictions(parameters: dict):
    predictions_api = "https://api.mosqlimate.org/api/registry/predictions/?"
    parameters_url = "&".join([f"{p}={v}" for p,v in parameters.items()])
    return requests.get(predictions_api + parameters_url).json()


In [6]:
def get_preds(parameters, geocode):
    
    res = get_predictions(parameters)
    
    df_in_json_format = res['items'][0]['prediction']
    json_struct = json.loads(df_in_json_format)    
    df_flat = pd.io.json.json_normalize(json_struct)
    df_flat.dates = pd.to_datetime(df_flat.dates)
    
    df_flat = df_flat.loc[df_flat.adm_2 == geocode]

    df_flat.dates = pd.to_datetime(df_flat.dates)
    
    df_flat['model'] = 'model id - '  + str(parameters['model_id'])
    
    return df_flat

    

In [7]:
geocode = 2304400

Previsões do modelo 4:

In [8]:
parameters = {
    "page": 1,
    "per_page": 50,
    "id": 8, 
    "model_id":4
    # Add parameters here
}

df_pred1 = get_preds(parameters, geocode = geocode)

df_pred1.head()

IndexError: list index out of range

Previsão do modelo 3

In [ ]:
parameters = {
    "page": 1,
    "per_page": 50,
    "id": 7, 
    "model_id":3
    # Add parameters here
}

df_pred2 = get_preds(parameters, geocode = geocode)

df_pred2.head()

In [ ]:
start = str(min(df_pred1.dates))[:10]
end = str(max(df_pred1.dates))[:10]

In [ ]:
url = "https://api.mosqlimate.org/api/datastore/infodengue/?"

parameters = {
        "per_page": 100,
        "disease": "dengue",
        "start": start,
        "end": end, 
        # Optional parameters are included here
        "geocode": geocode }
    
data = await get(url, parameters)
    
data = pd.DataFrame(data)
    
data = data[['data_iniSE', 'casos']]
    
df = data.rename(columns = {'data_iniSE':'dates', 'casos': 'target'})

df['legend'] = 'Data'

df.dates = pd.to_datetime(df.dates)

df = df.sort_values(by = 'dates')
df

### Criando o df_for:

In [ ]:
df_for = pd.concat([df_pred1, df_pred2], axis = 0)

df_for

In [ ]:
df_for.loc[df_for.model == 'model id - 4']

In [ ]:
df_for.dtypes

In [ ]:
# here is loaded the element that allows the selection by the mouse
highlight = alt.selection_point(on='mouseover', value = df_for.model.values[0], fields=['model'], nearest=True)

width = 375 # width of the plots

# here is loaded the data points (black)
data = alt.Chart(df).mark_circle(size = 60).encode(
    x='dates:T',
    y='target:Q',
    color=alt.value('black'),
    opacity=alt.Opacity('legend', legend=alt.Legend(title=None)),

    #size = alt.value(3)
    tooltip = 'target:Q'
).properties(
    width=width
)


# here is created the base element for the time series 
base = alt.Chart(df_for, title="Forecast of dengue new cases").encode(
   x=alt.X('dates:T').title('Dates'),
    y=alt.Y('preds:Q').title('New cases'),
    color='model:N'
).add_params(
    highlight
).properties(
    width=width
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_params(
    highlight
)


# here we create the multine plot and use the alt.condition to highlight only one curve
lines = base.mark_line().encode(
    #size=alt.condition(~highlight, alt.value(1), alt.value(3))
    color=alt.condition(highlight, alt.Color('model:N'), alt.value('lightgray')),
    tooltip = ['model:N', 'preds']
    
)

# here we define the plot of the right figure
timeseries = base.mark_line().encode(
    color=alt.Color('model:N')
).transform_filter(
    highlight # this function transform filter will just filter the element 
    #in hightlight from the column model N of the df_for (defined in the base element)
)

# here we create the area that represent the confidence interval of the predicitions
timeseries_conf = base.mark_area(
    opacity=0.5
).encode(
    x='dates:T',
    y='lower:Q',
    y2='upper:Q'
).transform_filter(
    highlight
)

# here we concatenate the layers, the + put one layer above the other
# the | put them syde by syde (as columns), and & put them side by side as lines
final = points + lines + data | timeseries + timeseries_conf + data

final

### Usando o plotly: 

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()


title = f"Forecasts at {geocode}"

fig.update_layout(width=900, height=500, title={
        'text': title,
        'y': 0.87,
        'x': 0.42,
        'xanchor': 'center',
        'yanchor': 'top'},
        xaxis_title='Date',
        yaxis_title=f'New cases',
        template='plotly_white')

# plot os dados
fig.add_trace(go.Scatter(x=df.dates, y=df.target,
                  name='Data', line=dict(color='black')))


# plot as previsões
fig.add_trace(go.Scatter(x=df_pred1.dates, y=df_pred1.preds, name='model id - 3',
                  line=dict(color='#FF7F0E')))

# plota os mínimos e máximos 
fig.add_trace(go.Scatter(x=df_pred1.dates, y=df_pred1.lower, line=dict(
        color='#FF7F0E', width=0), showlegend=False))

fig.add_trace(go.Scatter(x=df_pred1.dates, y=df_pred1.upper, line=dict(color='#FF7F0E', width=0),
                             mode='lines',
                             fillcolor='rgba(255, 127, 14, 0.3)', fill='tonexty', showlegend=False))



# plot as previsões
fig.add_trace(go.Scatter(x=df_pred2.dates, y=df_pred2.preds, name='model id - 4',
                  line=dict(color='#2CA02C')))

# plota os mínimos e máximos
fig.add_trace(go.Scatter(x=df_pred2.dates, y=df_pred2.lower, line=dict(
        color='#2CA02C', width=0), showlegend=False))

fig.add_trace(go.Scatter(x=df_pred1.dates, y=df_pred2.upper, line=dict(color='#2CA02C', width=0),
                             mode='lines',
                             fillcolor='rgba(44, 160, 44, 0.3)', fill='tonexty', showlegend=False))

    
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', zeroline=False,
                     showline=True, linewidth=1, linecolor='black', mirror=True)

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', zeroline=False,
                     showline=True, linewidth=1, linecolor='black', mirror=True)


